In [ ]:
import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix, classification_report, accuracy_score

import warnings
warnings.filterwarnings("ignore")

## Data Preparation

In [ ]:
dataset_path = "C:/Users/Public/AcademicProject/Clahe_WorkingDataset"
image_size = 224, 224
batch_size = 32

train_ds = image_dataset_from_directory(
  dataset_path,
  validation_split=0.2,
  subset="training",
  label_mode="categorical",
  seed=23,
  image_size=image_size,
  batch_size=batch_size)


val_ds = image_dataset_from_directory(
  dataset_path,
  validation_split=0.2,
  subset="validation",
  label_mode="categorical",
  seed=23,
  image_size=image_size,
  batch_size=batch_size)

In [ ]:
labels = train_ds.class_names
print(labels)

In [ ]:
# Data Augmentation

train_data_augmentation = Sequential([
    tf.keras.layers.Rescaling(1./255),             
    tf.keras.layers.RandomZoom(0.2),                
    tf.keras.layers.RandomFlip("horizontal"),        
    tf.keras.layers.RandomRotation(15),              
    tf.keras.layers.RandomWidth(0.1),                
    tf.keras.layers.RandomHeight(0.1),                
    tf.keras.layers.Resizing(224, 224)  
])

In [ ]:
val_data_augmentation = Sequential([
    tf.keras.layers.Rescaling(1./255),             
    tf.keras.layers.Resizing(224, 224)  
])

In [ ]:
train_dataset = train_ds.map(lambda x, y: (train_data_augmentation(x, training=True), y))

In [ ]:
val_dataset = val_ds.map(lambda x, y: (val_data_augmentation(x, training=True), y))

In [ ]:
val_batches = tf.data.experimental.cardinality(val_ds)
test_dataset = val_dataset.take(val_batches // 2)
val_dataset = val_dataset.skip(val_batches // 2)

In [ ]:
train_dataset = train_dataset.shuffle(buffer_size=1000)

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
val_dataset = val_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

## Model Training

In [ ]:
base_model = tf.keras.applications.DenseNet121(
    input_shape=(224, 224, 3), 
    include_top=False, 
    weights='imagenet')

# Freeze the base model
base_model.trainable = False
for layer in base_model.layers[-20:]:
    layer.trainable = True

# Build the model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),     
    BatchNormalization(),  
    Dropout(0.3),                 
    Dense(256, activation='relu'),
    BatchNormalization(),  
    Dropout(0.2),                  
    Dense(7, activation='softmax') 
])

model.compile(optimizer=Adam(learning_rate=0.001, 
          beta_1=0.9, beta_2=0.999, epsilon=1e-7),
          loss=CategoricalCrossentropy(),
          metrics=['accuracy'])

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=4, min_lr=1e-6)
best = ModelCheckpoint(filepath="densenet_2_model_best_val_accuracy.ckpt",
                                         save_weights_only=False,
                                         save_format = 'tf',
                                         monitor="val_accuracy",
                                         mode="max",
                                         save_best_only=True)

early_stop = EarlyStopping(monitor='val_loss', patience=15)

In [ ]:
# Save the model
model.save('densenet_2.h5')  # Save as .h5 file

# To reload it later
from tensorflow.keras.models import load_model
model = load_model('densenet_2.h5')

## Model Evaluation

In [ ]:
true_labels = []
predicted_labels = []

# Iterate through test dataset to get true labels
for images, labels in test_dataset:
    true_labels.extend(np.argmax(labels.numpy(), axis=1))  # Assuming one-hot encoded labels

    # Predict labels using the model
    predictions = model.predict(images)
    predicted_labels.extend(np.argmax(predictions, axis=1))

# Convert lists to NumPy arrays for easier processing
true_labels = np.array(true_labels)
predicted_labels = np.array(predicted_labels)

In [ ]:
accuracy = accuracy_score(true_labels, predicted_labels)
print(f"Accuracy: {accuracy:.2f}")

In [ ]:
loss, accuracy = model.evaluate(test_dataset)

print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

In [ ]:
from sklearn.metrics import classification_report, recall_score, f1_score,precision_score
# Recall
recall = recall_score(true_labels, predicted_labels, average='macro')
print(f"Recall: {recall:.2f}")
# Precision
precision = precision_score(true_labels, predicted_labels, average='macro')
print(f"Precision: {precision:.2f}")
# F1 Score
f1 = f1_score(true_labels, predicted_labels, average='macro')
print(f"F1 Score: {f1:.2f}")
# Detailed report (includes Precision, Recall, F1, and Accuracy)
report = classification_report(true_labels, predicted_labels, target_names=['angiodysplasia', 'esophagitis', 'normal-cecum', 'normal-pylorus', 'normal-z-line', 'polyps', 'ulcerative-colitis'])
print(report)

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

l = ['angiodysplasia', 'esophagitis', 'normal-cecum', 'normal-pylorus', 'normal-z-line', 'polyps', 'ulcerative-colitis']

cm = confusion_matrix(true_labels, predicted_labels)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=l, yticklabels=l)
plt.title("Confusion Matrix")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()